## Ridesharing Project

In [29]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import math
import os
from pymoo.core.problem import Problem, ElementwiseProblem
from pymoo.core.population import Population
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.optimize import minimize
from pymoo.core.repair import Repair
from pymoo.core.crossover import Crossover
from pymoo.core.mutation import Mutation
from pymoo.core.sampling import Sampling
from pymoo.core.selection import Selection
from pymoo.util.misc import random_permuations
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from pymoo.factory import get_reference_directions, get_selection
from pymoo.core.individual import Individual

In [33]:
# collect information about all instances
import os
pre_path = 'original_instances/'
re_path = 'instances/'
folders = os.listdir(pre_path)
folders.sort()
count = 0
ins_col = []
ins_to_use = []
for a_ins in folders:
    fre = a_ins.split('-')[-1].split('.')[0]
    inst = a_ins.replace('d6','').replace('d12','').replace('d24','')
    if inst in ins_col:
        continue
    count += 1
    ins_col.append(inst)
    ins_to_use.append(a_ins)
    
    file = pre_path + a_ins
    instance = open(file, 'r')
    head = [next(instance) for x in range(4)]
    name = head[0].replace('\n', '').split('-')
    road_map_code = name[1]
    if road_map_code == 'bj5':
        road_map = 'Beijing'
    elif road_map_code == 'cd1':
        road_map = 'Chengdu'
    else:
        road_map = 'Manhattan'
    capacity = name[3].replace('c', '')
    v_num = head[2].replace('\n', '').split(' ')[1]
    r_num = head[3].replace('\n', '').split(' ')[1]
    #print("instance" + str(count) + ": " + road_map + "\t" + v_num + '\t' + r_num)
    #print(str(count) + " & " + road_map + " & " + v_num + ' & ' + capacity + ' &  & ' + r_num + ' & ' + fre + ' &  \\\\')
    
    inst = pd.read_csv(file, skiprows = lambda x: x in range(6), sep = '\t', header=None)
    inst.columns = ['id', 's', 't', 'Q', 'time_s', 'time_t']
    inst.head(5)
    v_id = np.where(inst['Q'] < 0)[0]
    r_id = np.where(inst['Q'] > 0)[0]
    #print("Number of vehicles: ", len(v_id))
    #print("Number of riders: ", len(r_id))
    
    re_info = open(re_path + 'instance_'+ str(count) + '-info.txt', 'w')
    re_vehicles = re_path + 'instance_'+ str(count) + '-vehicles.csv'
    re_riders = re_path + 'instance_'+ str(count) + '-riders.csv'
    
    data = {
        "index": inst.iloc[r_id]['id'] - r_id[0],
        "s": inst.iloc[r_id]['s'],
        "t": inst.iloc[r_id]['t'],
        "et": inst.iloc[r_id]['time_s']
    }
    riders = pd.DataFrame(data=data)
    riders.to_csv(re_riders, index=False)
    
    data = {
        "index": inst.iloc[v_id]['id'],
        "s": inst.iloc[v_id]['s'],
        "t": inst.iloc[v_id]['s'], # since this data set does not set destination for vehicles, we assume they return to s
        "p": 0 - inst.iloc[v_id]['Q'],
        "e": list(map(lambda x: np.arange(1-x), inst.iloc[v_id]['Q'])),
        "c": 1
    }
    vehicles = pd.DataFrame(data=data)
    vehicles.to_csv(re_vehicles, index=False)
    
    info = road_map_code + '.edges\n' + re_riders + '\t' + r_num + '\n' + re_vehicles + '\t' + v_num + '\n'
    re_info.write(info)
    
# ins_to_use is the instances file for our experiments

In [6]:
# graph of the map
# load graph from a file
def map_graph(f_name):
    G = nx.Graph()
    edges = np.loadtxt(f_name, delimiter = " ", skiprows = 1)
    G.add_weighted_edges_from(edges)
    #G.add_weighted_edges_from([(1, 2, 12.5), (1, 3, 75), (2, 4, 29), (3, 4, 151)])
    return G
    
MG = map_graph("road-map/bj5.edges")

In [3]:
# split the riders and vehicles in a instance
inst = pd.read_csv("instances/rs-bj5-m1k-c1-d6-s10-x1.0.instance", 
                   skiprows = lambda x: x in range(6), sep = '\t', header=None)

inst.columns = ['id', 's', 't', 'Q', 'time_s', 'time_t']
inst.head(5)
v_id = np.where(inst['Q'] < 0)[0]
r_id = np.where(inst['Q'] > 0)[0]
print("Number of vehicles: ", len(v_id))
print("Number of riders: ", len(r_id))

Number of vehicles:  1000
Number of riders:  17467


In [4]:
# model of riders 
# file format: index s t et
def load_riders(f_name): 
    riders = pd.read_csv(f_name, sep= " ", header= None, index_col= 0)
    riders.columns=["index","s","t", "et"]
    return riders

riders = load_riders()
riders.head(3)

,index,s,t,et
1000,1,108984,71088,0
1001,2,9030,13449,0
1002,3,130576,333494,0


In [5]:
# vehicles
def load_vehicles(f_name):
    vehicles = pd.read_csv(f_name, sep= " ", header= None, index_col= 0)
    vehicles.columns=["index","s","t", "p", "e", "c"]
    return vehicles

vehicles = load_vehicles()
vehicles.head(3)

,index,s,t,p,e,c
0,1,32306,32306,1,"[0, 1]",1
1,2,98601,98601,1,"[0, 1]",1
2,3,143505,143505,1,"[0, 1]",1


# Define the problem
problem: find an arrangement for the riders and vehicles
solution: 3 x riders.shape[0]

In [6]:
class RideSharing(ElementwiseProblem):
    def __init__(self, mg, riders, vehicles):
        n_var = riders.shape[0]
        self.mg = mg
        self.speed = 2
        self.riders = riders
        self.vehicles = vehicles
        self.r = riders.shape[0]
        self.v = vehicles.shape[0]
        self.shape = np.array([self.r, 3])
        
        super().__init__(n_var=self.r, n_obj=6, n_constr=0, xl=0, xu=n_var-1)
    
    
    # emission calculation: vehicle v's emission of driving via a path
    def _emit(self, h, length, load):
        em_rate = h['e']
        pas = 0
        ems = 0
        for i in range(len(length)):
            cur_e = em_rate[pas]
            ems += cur_e * length[i]
            pas = load[i]
        
        return 0
    
    
    # calculate the gini coefficiency, 
    def _gini(self, a):
        a = a.flatten()
        if np.amin(a) < 0:
            a -= np.amin(a)
        a += 0.0000001
        a = np.sort(a)
        index = np.arange(1,a.shape[0]+1)
        n = a.shape[0]
        return ((np.sum((2 * index - n  - 1) * a)) / (n * np.sum(a)))
    
    
    def _evaluate(self, x, out, *args, **kwargs):
        # 1. get the arrangement for each vehicle 
        arr = x[:,0]
        travel_length = [] 
        travel_load = []
        travel_record = []
        travel_wait = []

        for i in range(self.v):
            vehicle = self.vehicles.iloc[i]
            pas_index = np.where(arr == i)[0]
            num = len(pas_index)
            record = np.zeros((num*2+2), dtype=int)
            record[0] = -1
            record[-1] = -1
            path = np.zeros((num*2+2), dtype=int)
            path[0] = vehicle['s']
            path[-1] = vehicle['t']
            pas = np.zeros((num*2+2), dtype=int)
            length = np.zeros((num*2+2), dtype=int)
            wait = np.zeros((num*2+2), dtype=int)

            board = dict()
            hold = dict()
            for i in pas_index:
                hold[i] = x[i]

            for j in range(1, len(path)-1, 1):
                off_weight = -1
                drop_rider = -1
                
                for key in board.keys():
                    if off_weight < board[key][2]:
                        off_weight = board[key][2]
                        drop_rider = key

                on_weight = -1
                pick_rider = -1
                for key in hold.keys():
                    if on_weight < hold[key][2]:
                        on_weight = hold[key][2]
                        pick_rider = key

                if pas[j-1] < vehicle['p']:
                    if off_weight >= on_weight:
                        path[j] = self.riders.iloc[drop_rider]['t']
                        board.pop(drop_rider, None)
                        pas[j] = pas[j-1] - 1
                        record[j] = drop_rider

                    else:
                        path[j] = self.riders.iloc[pick_rider]['s']
                        board[pick_rider] = x[pick_rider]
                        hold.pop(pick_rider, None)
                        pas[j] = pas[j-1] + 1
                        record[j] = pick_rider
                        
                        wait_time = np.sum(wait)
                        tra_dis = nx.dijkstra_path_length(self.mg, path[j-1], path[j])
                        tra_time = (np.sum(length) + tra_dis) / self.speed
                        wait[j] = self.riders.iloc[pick_rider]['et'] - (wait_time + tra_time) 
                        # >0, vehicle wait, < 0, rider wait

                else:
                    path[j] = self.riders.iloc[drop_rider]['t']
                    board.pop(drop_rider, None)
                    pas[j] = pas[j-1] - 1
                    record[j] = drop_rider

                length[j] = nx.dijkstra_path_length(self.mg, path[j-1], path[j])
            length[-1] = nx.dijkstra_path_length(self.mg, path[-2], path[-1])
            
            travel_record.append(record)
            travel_length.append(length) 
            travel_load.append(pas)
            travel_wait.append(wait)

        # 3. objectives
        # 1-EW, 2-ET
        ew = 0
        et = 0
        # and work time of vehicles, to include the time of waiting to pick up a rider into working time
        work_time = np.zeros((self.v), dtype = float)
        
        for i in range(self.r):
            driver = x[i][0]
            stops = np.where(travel_record[driver] == i)[0]
            wait = travel_wait[driver]
            s_stop = stops[0]
            t_stop = stops[1]
            if wait[s_stop] < 0:
                ew += 0 - wait[s_stop]
            else:
                work_time[driver] += wait[s_stop]
            
            travel_dis = np.sum(travel_length[driver][s_stop+1:t_stop+1])
            et += travel_dis / self.speed
        #end of for loop
        
        # 3-EC, 4-ED, 5-SE
        ec = 0
        ed = 0
        se = 0
        for i in range(self.v):
            h = self.vehicles.iloc[i]
            dis = np.sum(travel_length[i])
            work_time[i] += dis / self.speed
            ec += dis * h['c'] # assume here we have the cost per distance
            short_dis = nx.dijkstra_path_length(self.mg, h['s'], h['t'])
            ed += dis - short_dis
            se += self._emit(h, travel_length[i], travel_load[i])

        # 6-SG
        sg = self._gini(work_time)
        
        out["F"] = [ew, et, ec, ed, se, sg]
        


In [7]:
rs_problem = RideSharing(MG, riders, vehicles)
print(rs_problem.v)

1000


# Modefy NSGA3 
samiling, selection, crossover, mutation and elimination

In [8]:
# NSGA3 algorithm
init_size = 8
sele_rate = 0.5
offspring_size = init_size * sele_rate

In [9]:
# 1 define the initial populations
class RSSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        n_var = problem.riders.shape[0]
        speed = problem.speed
        r = problem.r
        v = problem.v
        pop = np.empty((0, r, 3), int)        
        
        for i in range(n_samples):            
            solution = np.empty((0,3), int)
    
            for i in range(r):
                chro = np.array([-1,-1,-1])
                driver = np.random.randint(v)
                chro[0] = driver
                chro[1] = 100 * np.random.random_sample()
                chro[2] = 100 * np.random.random_sample()
                solution = np.append(solution, [chro], axis=0)
            
            pop = np.append(pop, [solution], axis=0)
        
        print("Successfully init the samplings!")
        return pop

In [10]:
# 2 define the selection
class RSSelection(Selection):
    def _do(self, pop, n_select, n_parents, **kwargs):
        # number of random individuals needed
        n_random = n_select * n_parents
        n_perms = math.ceil(n_random / len(pop))
        P = random_permuations(n_perms, len(pop))[:n_random]
        X = np.reshape(P, (n_select, n_parents))
        return X

In [11]:
# 3 define the cross over
# crossover: two parents to one offspring
class RSCrossover(Crossover):
    def __init__(self, pattern_index):
        super().__init__(2, 1)
        self.pattern_index = pattern_index

    def _do(self, problem, X, **kwargs):
        shape = X.shape # n_offsprings, n_matings, problem.var
        _X = np.full((self.n_offsprings, shape[1], shape[2], shape[3]), int)
        Y = np.empty((0, shape[2], 3), int)
        parent_A = X[0]
        parent_B = X[1]
        for i in range(shape[1]):
            a_offspring = np.empty((0,3), int)
            A = parent_A[i]
            B = parent_B[i]
            parts = self.pattern_index + 1
            num = int(len(A) / parts) + 1
            for i in range(parts):
                from_p = num * i
                to_p = num * i + num
                if to_p > len(A):
                    to_p = len(A)
                if i % 2 == 0:
                    a_offspring = np.append(a_offspring, A[from_p:to_p, :], axis = 0)
                else:
                    a_offspring = np.append(a_offspring, B[from_p:to_p, :], axis = 0)
            Y = np.append(Y, [a_offspring], axis = 0)
        
        _X[0] = Y
        print("Successfully processed the crosscover!")
        return _X

In [12]:
# 4 define the mutation
# mutation: one riders change its drivers
class RSMutation(Mutation):
    def do(self, problem, X, **kwargs):
        Y = np.empty((0,3), int)
        for i in range(int(len(X)/2)):
            rider = X[i].X
            rider[0] = np.random.randint(problem.v)
            rider[1] += 100 * np.random.random_sample()
            rider[2] += 100 * ( np.random.random_sample() + 0.01 )
            
            Y = np.append(Y, [rider], axis = 0)
        Y = np.reshape(Y, (int(len(Y)/problem.r), problem.r, 3))
        print("Successfully processed the mutation!")  
        return Y

In [13]:
# Define the method to elimintae duplicated soltuions
class RS_eliminate_duplicates(ElementwiseDuplicateElimination):
    def is_equal(self, a, b):
        if isinstance(a, Individual):
            a_s = a.X
        else:
            a_s = a
            
        if isinstance(b, Individual):
            b_s = b.X
        else:
            b_s = b
        
        diff = len(np.where((a_s - b_s) != 0)[0])
        if diff > 0:
            return False
        else:
            return True
        
        return False
            
            
    def _do(self, pop, other, is_duplicate):
        def to_float(val):
            if isinstance(val, bool) or isinstance(val, np.bool_):
                return 0.0 if val else 1.0
            else:
                return val

        if other is None:
            for i in range(len(pop)):
                for j in range(i + 1, len(pop)):
                    val = self.cmp(pop[i], pop[j])
                    if val:
                        is_duplicate[i] = True
                        break
        else:
            for i in range(len(pop)):
                for j in range(len(other)):
                    val = self.cmp(pop[i], other[j])
                    if val:
                        is_duplicate[i] = True
                        break
        print("Successfully eliminated duplications")
        return is_duplicate

In [14]:
ref_dirs = get_reference_directions("das-dennis", 6, n_partitions=1)
rs_algorithm = NSGA3(pop_size = init_size,
                     ref_dirs = ref_dirs,
                     sampling = RSSampling(),
                     selection = RSSelection(),
                     crossover = RSCrossover(1),
                     mutation = RSMutation(),
                     eliminate_duplicates = RS_eliminate_duplicates(),
                     n_offsprings = offspring_size
                    )


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.show_compile_hint = False



In [15]:
# optimization and visualisation 
n_gen = 10
res = minimize(rs_problem,
               rs_algorithm,
               termination=('n_gen', n_gen),
               seed=1,
               save_history=True,
               verbose=True)
X = res.X
F = res.F
print(X)
print(F)

Successfully init the samplings!
Successfully eliminated duplications


KeyboardInterrupt: 

input part variation: Different type of riders:
(1) similar weight
(2) similar leaving time (maybe real-time)
(3) number of riders
(4) similar starting point
(5) similar/diversed destination

Different type of vehicles:
(1) vehicle capacities
(2) distribution of their starting points

Different rate of riders and vehicles

Alg Part variation: 
crossover and mutation

objectives variation/comparison from 3 aspects:
turn on or off the emission one